In [1]:
import pandas as pd

import numpy as np

import csv,sqlite3

In [2]:
conn = sqlite3.connect("predictions_sql.db")
cursor = conn.cursor()

In [3]:
df = pd.read_csv("predictions.csv", header=None, names=["user_id", "item_id", "rating_5"])

In [21]:
df.to_sql("predictions_tb", conn, index=False)

In [122]:
# read from sql db
# pred_df = pd.read_sql_query("select * from predictions_tb limit 5;", conn)

# pred_df

,user_id,item_id,rating_5
0,A2DJV5P2NTWEK0,0001981692,4.191430
1,A2DJV5P2NTWEK0,0002008211,4.699676
2,A2DJV5P2NTWEK0,0002254565,4.702403
3,A2DJV5P2NTWEK0,0002552493,4.645525
4,A2DJV5P2NTWEK0,0002710188,4.791867


In [4]:
# store pickle files
metadata = pd.read_pickle("refined_metadata.pkl")

metadata1 = metadata[['asin', 'title', 'authors', 'lexile']]

metadata1['lexile'] = pd.to_numeric(np.where(metadata1['lexile'].str[0].str.isalpha()\
                                             , metadata1['lexile'].str[2:-1], metadata1['lexile'].str[:-1]))

metadata1.head(10)

/home/apoorv/anaconda3/envs/cs670/lib/python2.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,asin,title,authors,lexile
0,0001006657,I Want My Dinner,"[{""name"":""Tony Ross"",""url"":""/content/scholasti...",240
1,0001981692,Pirates,"[{""name"":""Jim Brew"",""url"":""/content/scholastic...",770
2,0002008211,Haunted,"[{""name"":""R. L. Stine"",""url"":""/content/scholas...",460
3,0002199203,Rhino,"[{""name"":""Sheila Solomon Klass"",""url"":""/conten...",660
4,0002254565,Footsteps,"[{""name"":""Pamela Rushby"",""url"":""/content/schol...",260
5,0002552493,Hollywood Dogs,"[{""name"":""Meish Goldish"",""url"":""/content/schol...",720
6,0002710188,Home Run,"[{""name"":""Sara Hoskinson Frommer"",""url"":""/cont...",180
7,0003002241,Stuart Little,"[{""name"":""Garth Williams"",""url"":""/content/scho...",500
8,0006140882,Ransom,"[{""name"":""Saddleback Publishing"",""url"":""/conte...",340
9,0006137989,Crooked House,"[{""name"":""Agatha Christie"",""url"":""/content/sch...",630


In [160]:
cursor.execute('''DROP TABLE IF EXISTS metadata_tbl''')
conn.commit()

metadata1.to_sql("metadata_tbl", conn, index=False)

In [5]:
# store utility file
utility = pd.read_pickle("utility_df.pkl")

In [6]:
utility1 = utility.stack().to_frame("rating_5")

utility1.reset_index(level=1, inplace=True)
utility1.reset_index(level=0, inplace=True)

utility1.head(10)

,user,item,rating_5
0,A03208803KA5HCUYS17Z1,0062117378,5.0
1,A03208803KA5HCUYS17Z1,0062235656,5.0
2,A03208803KA5HCUYS17Z1,014241543X,5.0
3,A03208803KA5HCUYS17Z1,0142419400,4.0
4,A03208803KA5HCUYS17Z1,0374351384,3.0
5,A03208803KA5HCUYS17Z1,0375866566,2.0
6,A100YDQ9DIP06P,0060589507,5.0
7,A100YDQ9DIP06P,0061130990,3.0
8,A100YDQ9DIP06P,0061730831,5.0
9,A100YDQ9DIP06P,0061921033,5.0


In [111]:
cursor.execute('''DROP TABLE IF EXISTS utility_tbl''')
conn.commit()

utility1.to_sql("utility_tbl", conn, index=False)

In [7]:
# get user lexile rating
metadata2 = metadata1[["asin","lexile"]]

In [8]:
usr_lxl = utility1.merge(metadata2, how='left', left_on='item', right_on='asin')
usr_lxl.head(10)

,user,item,rating_5,asin,lexile
0,A03208803KA5HCUYS17Z1,0062117378,5.0,0062117378,610
1,A03208803KA5HCUYS17Z1,0062235656,5.0,0062235656,580
2,A03208803KA5HCUYS17Z1,014241543X,5.0,014241543X,830
3,A03208803KA5HCUYS17Z1,0142419400,4.0,0142419400,580
4,A03208803KA5HCUYS17Z1,0374351384,3.0,0374351384,720
5,A03208803KA5HCUYS17Z1,0375866566,2.0,0375866566,760
6,A100YDQ9DIP06P,0060589507,5.0,0060589507,150
7,A100YDQ9DIP06P,0061130990,3.0,0061130990,760
8,A100YDQ9DIP06P,0061730831,5.0,0061730831,490
9,A100YDQ9DIP06P,0061921033,5.0,0061921033,720


In [9]:
# We will require timestamp, otherwise hard to defend.
# sort usr_lxl at user, rating level & select first row for each user
usr_lxl2 = usr_lxl.sort_values(by=['user', 'rating_5'], ascending=False).groupby('user').first().reset_index()
usr_lxl3 = usr_lxl2[["user","lexile"]].rename(columns={'lexile':'lexile_user'})

usr_lxl3.head()

,user,lexile_user
0,A03208803KA5HCUYS17Z1,610
1,A100YDQ9DIP06P,150
2,A100ZQDV7L8PVV,1250
3,A104BOQOO5T2SU,430
4,A104SG9QZ92UAH,620


In [47]:
# merge user lexile ratings with predictions data & join lexile ratings of books
df2 = df.merge(usr_lxl3, how='left', left_on='user_id', right_on='user').drop('user',axis=1)\
        .merge(metadata1, how='left', left_on='item_id', right_on='asin')

df2.head()

,user_id,item_id,rating_5,lexile_user,asin,title,authors,lexile
0,A2DJV5P2NTWEK0,0001981692,4.191430,240,0001981692,Pirates,"[{""name"":""Jim Brew"",""url"":""/content/scholastic...",770
1,A2DJV5P2NTWEK0,0002008211,4.699676,240,0002008211,Haunted,"[{""name"":""R. L. Stine"",""url"":""/content/scholas...",460
2,A2DJV5P2NTWEK0,0002254565,4.702403,240,0002254565,Footsteps,"[{""name"":""Pamela Rushby"",""url"":""/content/schol...",260
3,A2DJV5P2NTWEK0,0002552493,4.645525,240,0002552493,Hollywood Dogs,"[{""name"":""Meish Goldish"",""url"":""/content/schol...",720
4,A2DJV5P2NTWEK0,0002710188,4.791867,240,0002710188,Home Run,"[{""name"":""Sara Hoskinson Frommer"",""url"":""/cont...",180


In [48]:
usr_itm_lxl = df2
usr_itm_lxl["lexile_diff"] = abs(usr_itm_lxl["lexile_user"] - usr_itm_lxl["lexile"])

In [49]:
usr_itm_lxl2 = usr_itm_lxl.drop(['asin'],axis=1).sort_values(by=['user_id', 'lexile_diff'], ascending=[False, True])

In [50]:
usr_itm_lxl2

,user_id,item_id,rating_5,lexile_user,title,authors,lexile,lexile_diff
2389317,AZXNB4Q5QCO2P,0060193263,4.058366,1050,Cowboy,"[{""name"":""Robert Klausmeier"",""url"":""/content/s...",1050,0
2389627,AZXNB4Q5QCO2P,0061240273,3.918141,1050,Belong to Me,"[{""name"":""Marisa De los Santos"",""url"":""/conten...",1050,0
2390561,AZXNB4Q5QCO2P,030726579X,3.994089,1050,Heart and Soul,"[{""name"":""Kadir Nelson"",""url"":""/content/schola...",1050,0
2390691,AZXNB4Q5QCO2P,031259612X,3.929125,1050,Wereling,"[{""name"":""Steve Feasey"",""url"":""/content/schola...",1050,0
2390741,AZXNB4Q5QCO2P,0316005363,4.074077,1050,Matter,"[{""name"":""Christopher Cooper"",""url"":""/content/...",1050,0
2390756,AZXNB4Q5QCO2P,0316040096,3.905804,1050,Ash,"[{""name"":""Malinda Lo"",""url"":""/content/scholast...",1050,0
2391574,AZXNB4Q5QCO2P,0399234764,4.254070,1050,The Cod's Tale,"[{""name"":""Mark Kurlansky"",""url"":""/content/scho...",1050,0
2391692,AZXNB4Q5QCO2P,0425209857,4.254094,1050,French Kiss,"[{""name"":""Aimee Friedman"",""url"":""/content/scho...",1050,0
2391707,AZXNB4Q5QCO2P,0439042968,4.411782,1050,882 1/2 Amazing Answers to Your Questions Abou...,"[{""name"":""Ken Marschall"",""url"":""/content/schol...",1050,0
2392085,AZXNB4Q5QCO2P,0446345342,4.263106,1050,Underworld,"[{""name"":""James Gulliver Hancock"",""url"":""/cont...",1050,0


In [51]:
# distinct users
len(usr_itm_lxl2["user_id"].unique())

3822

In [52]:
usr_itm_lxl2[usr_itm_lxl2["rating_5"]>4.75].count()

user_id        1402231
item_id        1402231
rating_5       1402231
lexile_user    1402231
title          1402231
authors        1402231
lexile         1402231
lexile_diff    1402231
dtype: int64

In [53]:
usr_itm_lxl3 = usr_itm_lxl2[usr_itm_lxl2["rating_5"]>4.75]

usr_itm_lxl3.head(10)

,user_id,item_id,rating_5,lexile_user,title,authors,lexile,lexile_diff
8796540,AZWC9XAY34IPW,0140501738,4.756141,600,Corduroy,"[{""name"":""Don Freeman"",""url"":""/content/scholas...",600,0
8796769,AZWC9XAY34IPW,0142501123,4.930804,600,Room on the Broom,"[{""name"":""Axel Scheffler"",""url"":""/content/scho...",600,0
8796964,AZWC9XAY34IPW,015216748X,4.776133,600,Epossumondas,"[{""name"":""Coleen Salley"",""url"":""/content/schol...",600,0
8798009,AZWC9XAY34IPW,0399226710,4.770887,600,Pink and Say,"[{""name"":""Patricia Polacco"",""url"":""/content/sc...",600,0
8799654,AZWC9XAY34IPW,0688128971,4.918088,600,Lilly's Purple Plastic Purse,"[{""name"":""Kevin Henkes"",""url"":""/content/schola...",600,0
8799978,AZWC9XAY34IPW,0738732516,4.820977,600,Beautiful Music for Ugly Children,"[{""name"":""Kirstin Cronn-Mills"",""url"":""/content...",600,0
8797597,AZWC9XAY34IPW,0375863400,4.923251,600,Split,"[{""name"":""Swati Avasthi"",""url"":""/content/schol...",610,10
8797842,AZWC9XAY34IPW,0394810406,4.852140,600,Frederick,"[{""name"":""Leo Lionni"",""url"":""/content/scholast...",590,10
8798490,AZWC9XAY34IPW,0440417996,4.756408,600,The Name Jar,"[{""name"":""Yangsook Choi"",""url"":""/content/schol...",590,10
8798494,AZWC9XAY34IPW,0440419441,4.756099,600,When My Name Was Keoko,"[{""name"":""Linda Sue Park"",""url"":""/content/scho...",610,10


In [55]:
# At limit of rating = 4.75, a total of 275 (out of 3800) users have less
# than 10 recommended ratings
usr_itm_lxl3.groupby('user_id').count().sort_values(by='item_id').head(280)

,item_id,rating_5,lexile_user,title,authors,lexile,lexile_diff
user_id,,,,,,,
A20CPKNQU0QE97,1,1,1,1,1,1,1
A3Z9QN0RCOHEY,1,1,1,1,1,1,1
AX7QMRXX81L9K,1,1,1,1,1,1,1
AC0WM9BML7FPT,1,1,1,1,1,1,1
A26N37CPSDWD8D,1,1,1,1,1,1,1
A1EKF9JFF1XWSV,1,1,1,1,1,1,1
AOLRFN1T5BCGE,1,1,1,1,1,1,1
A1R0M8JWQL95W8,1,1,1,1,1,1,1
A30YCQ4MOZAI6Y,1,1,1,1,1,1,1
